# ROMA & RBAC - Intro to SQL

## 1 Preparation

We start by setting up a connection to our sample database (DB), Northwind DB. The preparation consists of the following steps.

1. Install a few Python packages to faciliate DB connection and further analysis via SQL.

2. Import the packages and configure a few settings.

3. Initialize a DB connection.

4. Import data and create DB tables. (We are in fact building up a DB.)

Normally setting up a DB system would be the task of a DB engineer, and it's a complicated process. As a data/business analyst, you simply connect to a DB, query the data stored in DB tables, and perform various analysis. However, to prepare you for the case competition, I'll show you how to setup a lightweight DB system on your own, and import raw data to create DB tables.

The choice of our database engine is [DuckDB](https://duckdb.org/). DuckDB is an in-process OLAP Relational DB Management System (RDBMS). In-process DBMS, sometimes also called embedded DBMS, means it's lightweight and can be easily integrated with your application. OLAP (OnLine Analytical Processing) means the DBMS is desinged mainly to support analytical query workloads, which include complex long-running queries, aggregations and table joins.

DuckDB comes with many cool features, in particular, it's columnar-vectorized query execution engine. I won't get into the details. See [here](https://duckdb.org/why_duckdb.html) if you are interested.

Typical enterprise-level RDBMS (MS SQL server, PostgreSQL, etc.) are much more complicated to setup and manage. DuckDB is excellent for stand-alone data analytics tasks on your local computer, and a good DBMS to learn SQL. ([SQLite](https://www.sqlite.org/index.html) is another good choice.)

However, I also want to point out that DuckDB is only a few years old (preview  released in 2019), and it's still in active development and is maintained by the open source community, so you may encounter minor bugs when using it.

### 1.1 Insall Python packages

In [ ]:
# 1) duckdb: DuckDB Python API
# ref: https://duckdb.org/docs/api/python/overview.html

# 2) jupysql: run SQL and plot data in a Jupyter notebook via %sql, %%sql, and %sqlplot magics
# ref: https://jupysql.readthedocs.io/en/latest/quick-start.html

# 3) duckdb-engine: Basic SQLAlchemy driver for DuckDB
# ref: https://github.com/Mause/duckdb_engine

# 4) pandas: Python dataframe manipulation tool
# ref: https://pandas.pydata.org/

# 4) matplotlib: Python plotting tool
# ref: https://matplotlib.org/

!pip install --quiet duckdb 
!pip install --quiet jupysql 
!pip install --quiet duckdb-engine
!pip install --quiet pandas
!pip install --quiet matplotlib

### 1.2 Import the packages and configure a few settings

In [ ]:
import duckdb
import pandas as pd
# No need to import sqlalchemy or duckdb_engine
# JupySQL will use SQLAlchemy to auto-detect the driver needed based on your connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

Configure jupysql to return data as a Pandas dataframe and have less verbose output

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

### 1.3 Connecting to DuckDB
Connect jupysql to DuckDB using a SQLAlchemy-style connection string. You may either connect to an in memory DuckDB, which is our case, or a file backed DB.

In [ ]:
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

Once the connection is set, let's check the version of DuckDB we are using.

In [ ]:
%%sql
-- check DuckDB version
PRAGMA version;

### 1.4 Download raw data and create tables in the Database

In [ ]:
# download raw data for the Northwind DB: 8 csv data files stored on github
!wget --quiet https://raw.githubusercontent.com/tdmdal/datasets-teaching/main/nw/csv/Categories.csv
!wget --quiet https://raw.githubusercontent.com/tdmdal/datasets-teaching/main/nw/csv/Customers.csv
!wget --quiet https://raw.githubusercontent.com/tdmdal/datasets-teaching/main/nw/csv/Employees.csv
!wget --quiet https://raw.githubusercontent.com/tdmdal/datasets-teaching/main/nw/csv/OrderDetails.csv
!wget --quiet https://raw.githubusercontent.com/tdmdal/datasets-teaching/main/nw/csv/Orders.csv
!wget --quiet https://raw.githubusercontent.com/tdmdal/datasets-teaching/main/nw/csv/Products.csv
!wget --quiet https://raw.githubusercontent.com/tdmdal/datasets-teaching/main/nw/csv/Shippers.csv
!wget --quiet https://raw.githubusercontent.com/tdmdal/datasets-teaching/main/nw/csv/Suppliers.csv

In [ ]:
# import csv files to create tables
# ref: https://duckdb.org/docs/data/csv.html
%%sql
DROP TABLE IF EXISTS Categories; 
CREATE TABLE Categories AS SELECT * FROM read_csv_auto('Categories.csv', NULLSTR="NULL");

DROP TABLE IF EXISTS Customers; 
CREATE TABLE Customers AS SELECT * FROM read_csv_auto('Customers.csv', NULLSTR="NULL", HEADER=TRUE);

DROP TABLE IF EXISTS Employees; 
CREATE TABLE Employees AS SELECT * FROM read_csv_auto('Employees.csv', NULLSTR="NULL");

DROP TABLE IF EXISTS OrderDetails; 
CREATE TABLE OrderDetails AS SELECT * FROM read_csv_auto('OrderDetails.csv', NULLSTR="NULL");

DROP TABLE IF EXISTS Orders; 
CREATE TABLE Orders AS SELECT * FROM read_csv_auto('Orders.csv', NULLSTR="NULL");

DROP TABLE IF EXISTS Products; 
CREATE TABLE Products AS SELECT * FROM read_csv_auto('Products.csv', NULLSTR="NULL");

DROP TABLE IF EXISTS Shippers; 
CREATE TABLE Shippers AS SELECT * FROM read_csv_auto('Shippers.csv', NULLSTR="NULL");

DROP TABLE IF EXISTS Suppliers;
CREATE TABLE Suppliers AS SELECT * FROM read_csv_auto('Suppliers.csv', NULLSTR="NULL");

In the above, we first drop (delete) a table if it already exists in order to create a fresh new table of the same name. We then create the table, whose content is the the result of selecing all columns (`SELECT * FROM`) of the imported csv file (`read_csv_auto()`).

`read_csv_auto()` imports a CSV file by automatically analyzing the file to decide how to parse it, i.e., to determine whether the file has a header row, what's the type of each data column, etc. Details of `read_csv_auto()` parameters/configurations can be found [here](https://duckdb.org/docs/data/csv). After you import a dataset and create a table for it, you always want to verify whether the data import process is done correctly. See Section 2.1, 2.2, and Ex.1 for how to do that.

Note that we won't further establish relationships (primary keys, foreign keys, constraints, etc.) between tables. This is for simplicity. After all, this is an intro workshop.

## 2 A quick look at Northwind DB

### 2.1 How many tables does this DB have?

A database may contain many tables. In our case, we just imported 8 raw CSV files and created 8 tables in the database, so let's confirm it.

In [ ]:
%%sql
PRAGMA show_tables;

### 2.2 How does the Categories table look like?

A table is made up of one or more columns/fields. Let's take a look a the Categories table (column names, types, etc.).

In [ ]:
%sql PRAGMA table_info("Categories");

A column/field can be of different types, i.e., it can store different types of data. In DuckDB, for example, the [types](https://duckdb.org/docs/sql/data_types/overview.html) can be INTEGER, REAL, TIMESTAMP, etc. 

### 2.3 How about the relationships between the tables?

We present the relationships (primary keys, foreign keys, constraints, etc.) between the tables using an Entity Relationship (ER) diagram.

Note that for a well-designed and maintained DB, a DB engineer would establish those relationships between the tables, and enforce them in the DB system. In our case, after we imported data and created the tables, we didn't create any relationships. (This is an intro workshop so we kept things simple.) Therefore, the relationships presented in the ER diagram are more of "virtual" relationships, i.e., logical relations exist in your head.

<img src="https://github.com/tdmdal/datasets-teaching/raw/main/nw/images/NW_ER.png" alt="ER Diagram" width="836" height="605" />

In the above ER diagram, the tiny vertical key icon indicates a column is a primary key. A primary key is a column (or set of columns) whose values uniquely identify every row in a table. For example, `OrderID` is the primary key in the `Orders` table, and `OrderID` and `ProductID` (combined) is the primary key in the `OrderDetails` table.

The relationship icon (a line with a horizontal key at one end and an infinite symbol at the other end) indicates a foreign key constraint and a one-to-many relationship. A foreign key is a column (or set of columns) in one table whose values uniquely identify a row of another table or the same table. A foreign key mostly refers to a primary key in another table. A foreign key constraint requires that the constrained column contain only values from the primary key column of the other table. For example `CustomerID` in the `Orders` table is a foreign key that refers to the `CustomerID` primary key in the `Customers` table, and it can only contain values that exist in the `CustomerID` column of the `Customers` table.

In addition, it happens that every foreign key constraint in the Northwind DB establishes a one-to-many relationship, i.e. a row from one table can have multiple matching rows in another table. For example, one row from the `Customers` table can match multiple rows in the `Orders` table (via `CustomerID`). This makes sense as one customer can place more than one orders. (Another common relationship a foreign key constraint can establish is the one-to-one relationship.)

|logo|meaning|
|:------:|:------:|
|![key logo](https://github.com/tdmdal/datasets-teaching/raw/main/nw/images/key_vertical.png "key logo")|primary key|
|![foreign key constraint](https://github.com/tdmdal/datasets-teaching/raw/main/nw/images/relationship.png "foreign key constraint")|one-to-many foreign key contraint|

**Question**: Why do we need foreign key constraints? (Discussion)

## 3 Learning SQL basics by doing

**We will work on the problems with a * sign together.** Other problems are left for homework.

### Part 1. Basics

We will learn

1. Retrieve data: `SELECT...FROM...`
2. Sort retrieved data: `SELECT...FROM...ORDER BY...`
3. Filter data: `SELECT...FROM...WHERE...`; `IN`, `NOT`, `LIKE` and `%` wildcard
4. Create calculated columns (fields): mathematical calculations (e.g. +, -, *, /); data manipulation functions (e.g. `age()`, `||`) 

#### Ex1*. Which categories do we have?

Display `CategoryName` and `Description` in the `Categories` table.

In [ ]:
%%sql
-- start your code here


Note that a query result can be saved into a Python dataframe using the `<<` operator. You can further process the dataframe using Python, or export it as a CSV file using `your_dataframe.to_csv('your_filename.csv')` to further process it in other analytics tools.

In [ ]:
# demo code here: %sql my_df <<
# %sql my_df << ;

In [ ]:
# demo code here: %%sql my_df <<
# %%sql
# my_df << ;

In [ ]:
# demo code here: my_df
# my_df

In [ ]:
 # demo code here:
 # my_df.to_csv("my_data.csv")

Alternatively, you can also create a temporary table from the `SELECT` output, and further query it using SQL.

In [ ]:
%%sql
-- demo code here
-- DROP TABLE IF EXISTS my_category; 

-- CREATE TEMP TABLE my_category AS

-- SELECT * FROM my_category;

#### Ex2. Which shippers do we have?

In [ ]:
%%sql
-- start your code here


#### Ex3* Find names and hire dates of all US sales representatives

The resulting table should display `FirstName`, `LastName`, `HireDate`, `Title` and `Country`.

In [ ]:
%%sql
-- take a look at the Employees table first
-- use LIMIT to limit the rows being displayed
SELECT FirstName, LastName, HireDate, Title, Country
FROM Employees
LIMIT 10;

In [ ]:
%%sql
-- just curious, how many distinct coutries do we have?
SELECT DISTINCT Country
FROM Employees;

In [ ]:
%%sql
-- start your code here


#### Ex4. Find supplier contacts who are *not* marketing managers

Display `supplierID`, `ContactNames` and `ContactTitle` for those whose `ContactTitle` is *not* Marketing Manager.

Hint: `NOT` operator. See [here](https://duckdb.org/docs/sql/expressions/logical_operators).

In [ ]:
%%sql
-- start your code here


#### Ex.5* Find products with "Chef" in its name

Hint: `LIKE` operator and `%` wildcard. See [here](https://duckdb.org/docs/sql/functions/patternmatching.html) for details.

In [ ]:
%%sql
-- start your code here


#### Ex.6 Find all orders shipping to Canada, Mexico or USA.

Hint: `IN` operator. See [here](https://duckdb.org/docs/sql/expressions/in.html).

In [ ]:
%%sql
-- start your code here


#### Ex.7* Who is the oldest employee?

Display `FirstName`, `LastName`, `Title`, `BirthDate`, and age. Order the result by age in descending order.

Hint: [`age()`](https://duckdb.org/docs/sql/functions/timestamp) and [`year()`](https://duckdb.org/docs/sql/functions/datepart) functions.

In [ ]:
%%sql
-- start your code here


#### EX.8 Show employee full name

Show the `FirstName` and `LastName` columns from the `Employees` table, and then create a new column called `FullName`, showing first name and last name concatenated in the format of "LASTNAME, Firstname", e.g. "DAVOLIO, Nancy".

Hint: See [text functions](https://duckdb.org/docs/sql/functions/char).

In [ ]:
%%sql
-- start your code here


#### Ex.9* Which order items have total sales above $12,000.

In the `OrderDetails` table, we have the columns `UnitPrice` and `Quantity`. Create a new column, `TotalPrice`, that multiplies these two (ignore the `Discount` column). Display `OrderID`, `ProductID`, `UnitPrice`, `Quantity` and `TotalPrice`. Order the result first by `ProductID` (ascending) and then by `TotalPrice` (descending).

Note that the question asks about order items (not orders). One order may contain multiple items/products. An order item is identified by the `OrderID` and `ProductID` columns.

In [ ]:
%%sql
-- start your code here


#### Ex.10 Show all products that need reordering

A product need reordering if `UnitsInStock + UnitsOnOrder <= ReorderLevel` and `Discontinued = 0`. In your result, display `ProductID`, `ProductName`, `UnitsInStock`, `UnitsOnOrder`, `ReorderLevel`, and `Discontinued`.

In [ ]:
%%sql
-- start your code here


### Part 2. Summarize and Group Data

1. Summarize data using aggregate functions (e.g. `COUNT()`, `MIN()`, `MAX()`, and `AVG()`).
2. Group data and filter groups: `SELECT...FROM...GROUP BY...HAVING...`.
3. `SELECT` clause ordering: `SELECT...FROM...WHERE...GROUP BY...HAVING...ORDER BY...`.
4. Filter data by *subquery* (optional): `SELECT...FROM...WHERE...(SELECT...FROM...)`

#### Ex.11* Count total number of order items. What's their minimum, maximum and average price?

Note that the question asks about order items (not orders). One order may contain multiple items/products. An order item is identified by (`OrderID` + `ProductID`).

In [ ]:
%%sql
-- start your code here


#### Ex. 12* How many customers do we have for each contact title.

Order the result by the count of contact titles.

In [ ]:
%%sql
-- start your code here


#### Ex.13 Find total customers per country and city

In [ ]:
%%sql
-- start your code here


#### Ex.14* Show all orders with values greater than $12,000?

Ignore the `Discount`. Contrast this question with Ex.9.

In [ ]:
%%sql
-- start your code here


#### Ex.15* (Optional) Find top three countries with the highest average freight charges for the last 12 months. 

"The last 12 months" is with respect to the last `OrderDate` in the `Orders` table.

In [ ]:
%%sql
-- start your code here


### Part 3. Join Tables

1. Inner join: `SELECT...FROM...INNER JOIN...ON...`
2. Left join:  `SELECT...FROM...LEFT JOIN...ON...`
3. Other joins.

#### Ex.16* Show products and their associated suppliers

Display the `ProductID`, `ProductName`, and the `CompanyName` of the Supplier. Sort by `ProductID`.

We can use the `WHERE` keyword.

In [ ]:
%%sql
-- start your code here (WHERE keyword)
SELECT ProductID, ProductName, Products.SupplierID, CompanyName as Supplier
FROM Products, Suppliers
WHERE Products.SupplierID = Suppliers.SupplierID
ORDER BY ProductID;

We can also use the `(INNER) JOIN` keyword.

In [ ]:
%%sql
-- start your code here (JOIN keyword)


#### Ex.17 Show all orders and shippers with OrderID less than 10255.

Display `OrderID`, `OrderDate` (date part only), and `CompanyName` of the Shipper, and sort by `OrderID`. In addition, only display rows with `OrderID < 10255`.

In [ ]:
%%sql
-- start your code here


#### Ex.18* Find total number of products in each category

In your result, display `CategoryName` and total number of product.

In [ ]:
%%sql
-- start your code here


#### Ex.19 Show all orders with values greater than $12,000 and are placed in 2016?

In your result, display `OrderID` and total value of the order (ignore `Discount`).

In [ ]:
%%sql
-- start your code here


#### Ex.20* What products has Michael Suyama ever sold?

In your result, display `EmployeeID`, `FirstName`, `LastName`, `ProductID` and `ProductName`.

In [ ]:
%%sql
-- start your code here


#### Ex.21* Find customers that never placed an order

In [ ]:
%%sql
-- start your code here


Note that the Jupyter Notebook displays Null as None.

## 4 Summary

We have covered quite a few basic SQL keywords. They are mostly about getting data from a DB and manipulate the retrieved data. There are much more to SQL. Take a look at this [post](https://stackoverflow.com/questions/2054130/what-is-advanced-sql) to get an idea of what advanced SQL topics are about, and if you are interested, explore them on your own.

## 5 References

Most of the exercises in this notebook are drawn from this [book](https://www.amazon.ca/SQL-Practice-Problems-learn-doing-ebook/dp/B01N41VQFO) by Sylvia Moestl Vasilik. The book is based on Microsoft SQL server and its T-SQL.

Another great SQL book for beginner is [SQL in 10 Minute](https://www.amazon.ca/SQL-Minutes-Sams-Teach-Yourself/dp/0135182794) by Ben Forta.

## Appendix

Examples of various joins shown in the slides.

In [ ]:
# create a new duckdb database
%sql duckdb:///play_ground.duckdb;

In [ ]:
%%sql

-- create left table: table1
DROP TABLE IF EXISTS table1;

CREATE TABLE table1 (
  pk INTEGER NOT NULL,
  t1c1 TEXT NOT NULL
);

INSERT INTO table1 
    (pk, t1c1) 
VALUES 
    (1, 'a'),
    (2, 'b');

-- create right table: table2
DROP TABLE IF EXISTS table2;

CREATE TABLE table2 (
  fk INTEGER NOT NULL,
  t2c1 TEXT NOT NULL
);

INSERT INTO table2 
    (fk, t2c1) 
VALUES 
    (1, 'c'),
    (1, 'd'),
    (3, 'e');

In [ ]:
%%sql
SELECT * FROM table1;

In [ ]:
%%sql
SELECT * FROM table2;

In [ ]:
%%sql
-- inner join
SELECT * FROM table1 JOIN table2 ON table1.pk = table2.fk;

In [ ]:
%%sql
-- left join
SELECT * 
FROM table1 
  LEFT JOIN table2 ON table1.pk = table2.fk;

In [ ]:
%%sql
-- left join with exclusion (using LEFT JOIN and WHERE)
SELECT * 
FROM table1
  LEFT JOIN table2 ON table1.pk = table2.fk
WHERE table2.fk IS NULL;

In [ ]:
%%sql
-- right join using RIGHT JOIN key word
SELECT * 
FROM table1
  RIGHT JOIN table2 ON table2.fk = table1.pk;

In [ ]:
%%sql
-- right join with exclusion using RIGHT JOIN and WHERE key word
SELECT * 
FROM table1
  RIGHT JOIN table2 ON table2.fk = table1.pk
WHERE table1.pk is NULL;

In [ ]:
%%sql
-- fuller outer join
SELECT *
FROM table1
  FULL JOIN table2 ON table1.pk = table2.fk;

In [ ]:
%%sql
-- full outer join with exclusion
SELECT *
FROM table1
  FULL JOIN table2 ON table1.pk = table2.fk
WHERE table1.pk is NULL OR table2.fk is NULL;